In [2]:
from scipy.sparse import csr_matrix

import seml
import pandas as pd
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [5]:
results = seml.get_results("moscot_map", to_data_frame=True)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
results

,_id,config.overwrite,config.db_collection,config.dataset,config.method,config.params.alpha,config.params.epsilon,config.path_data,config.path_results,config.seed,...,result.gene_size,result.corr_results_mean,result.corr_results_median,result.corr_results_var,result.corr_results_max,result.corr_results_min,config.params.mode,result.method_info.kl_reg,config.params.epochs,result.method_info
0,1,1,moscot_map,45,MOSCOT,0.1,0.01,/lustre/groups/ml01/workspace/giovanni.palla/m...,/lustre/groups/ml01/workspace/moscot_paper/ben...,1,...,100,pearson 0.004086 spearman 0.009048 dtyp...,pearson -0.004931 spearman 0.008378 dtyp...,pearson 0.002261 spearman 0.002068 dtyp...,pearson 0.345273 spearman 0.343883 dtyp...,pearson -0.044990 spearman -0.062692 dtyp...,NaN,NaN,NaN,NaN
1,2,2,moscot_map,45,TANGRAM,NaN,NaN,/lustre/groups/ml01/workspace/giovanni.palla/m...,/lustre/groups/ml01/workspace/moscot_paper/ben...,1,...,100,pearson 0.020251 spearman 0.028272 dtyp...,pearson 0.012475 spearman 0.028542 dtyp...,pearson 0.002702 spearman 0.002618 dtyp...,pearson 0.122622 spearman 0.142128 dtyp...,pearson -0.296694 spearman -0.257545 dtyp...,cluster,3.201991916057523e-08,NaN,NaN
2,3,3,moscot_map,45,GIMVI,NaN,NaN,/lustre/groups/ml01/workspace/giovanni.palla/m...,/lustre/groups/ml01/workspace/moscot_paper/ben...,1,...,100,pearson 0.008628 spearman 0.009173 dtyp...,pearson 0.006998 spearman 0.006489 dtyp...,pearson 0.001481 spearman 0.001461 dtyp...,pearson 0.146022 spearman 0.104227 dtyp...,pearson -0.097984 spearman -0.113061 dtyp...,NaN,NaN,10.0,NaN
